In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Importing packages

In [ ]:
import pandas as pd
import networkx as nx

In [ ]:
# !ls "/content/drive/MyDrive/Network Science Project Files"


In [ ]:
file_path = "/content/drive/MyDrive/Network Science Project Files"

In [ ]:
df = pd.read_csv(file_path+'/all_records.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (20,21,22,32) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# df.head()

In [ ]:
df.columns = list(map(str.strip, df.columns))

In [ ]:
df['NORMED?'] = df['NORMED?'].str.strip()
df['CUE'] = df['CUE'].str.strip()
df['TARGET'] = df['TARGET'].str.strip()
df['TPS'] = df['TPS'].str.strip()
df['QPS'] = df['QPS'].str.strip()

In [ ]:
df = df[df['NORMED?']=='YES']

In [ ]:
df = df[['CUE','TARGET','NORMED?','#G','#P','FSG','BSG','QPS','TPS']]

* CUE	Normed Word
* TARGET	Response to Normed Word
* NORMED?	Is Response Normed?
* \#G	Group size
* \#P	Number of Participants Producing Response
* FSG	Forward Cue-to-Target Strength
* BSG	Backward Target-to-Cue Strength
* QPS	Cue: Part of Speech
* TPS	Target: Part of Speech





In [ ]:
## filtering POS for CUES and TARGETS to be, nouns verbs and adjectives
exclude_list = ['PP','AV','ADV','P','I','C','AD']
df1 = df[~df['QPS'].str.contains('|'.join(exclude_list))]
df1['TPS'] = df1['TPS'].dropna()
df1 = df1[df1['TPS']!=' ']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


* N	Noun
* V	Verb
* AJ	Adjective
* AD	Adverb
* P	Pronoun
* PP	Preposition
* I	Interjection
* C	Conjunction

In [ ]:
df1['BSG'] = df1['BSG'].astype(float)

Filtering BSG to be 0 and the number of participants to be more than 2

In [ ]:
df1 = df1[df1['BSG']==0]
df1 = df1[df1['#P']>2]

## Graph for the Cues starting with letter A

In [ ]:
# df1_n_bsg_a_b = df1_n_bsg[(df1_n_bsg['CUE'].str.startswith('A')) | (df1_n_bsg['CUE'].str.startswith('B'))]
df1_n_bsg_a = df1[(df1['CUE'].str.startswith('A'))]

In [ ]:
lst = list(set(list(df1_n_bsg_a['CUE'])+list(df1_n_bsg_a['TARGET'])))

In [ ]:
# G = nx.Graph()
# for n in lst:
#   G.add_node(n)
# for row in df1_n_bsg_a.itertuples():
#   # print(row.CUE)
#   G.add_edge(row.CUE,row.TARGET)

In [ ]:
nx.write_gml(G,'cue_target.gml')

## Generating Graph for the the Parts of Speech of the Cues and the Target - Cues are already filtered for Nouns, Verbs and Adjectives

In [ ]:
df1

,CUE,TARGET,NORMED?,#G,#P,FSG,BSG,QPS,TPS
4,A,LETTER,YES,152,6,0.039,0.0,N,N
5,A,PLUS,YES,152,5,0.033,0.0,N,PP
13,AARDVARK,ANIMAL,YES,152,49,0.322,0.0,N,N
14,AARDVARK,ANT,YES,152,30,0.197,0.0,N,N
17,AARDVARK,MUSIC,YES,152,3,0.020,0.0,N,N
...,...,...,...,...,...,...,...,...,...
63812,ZUCCHINI,VEGETABLE,YES,148,49,0.331,0.0,N,N
63813,ZUCCHINI,GREEN,YES,148,22,0.149,0.0,N,N
63815,ZUCCHINI,FOOD,YES,148,13,0.088,0.0,N,N
63816,ZUCCHINI,BROCCOLI,YES,148,5,0.034,0.0,N,N


In [ ]:
df1.dtypes

CUE         object
TARGET      object
NORMED?     object
#G           int64
#P           int64
FSG        float64
BSG        float64
QPS         object
TPS         object
dtype: object

In [ ]:
df1 = df1[df1['TPS']!='']

In [ ]:
df1['TPS'].value_counts()

N      16739
AJ      4321
V       4047
AD       425
P         75
ADJ       71
PP        55
I         33
A          6
INT        4
AV         3
PRP        2
C          2
Name: TPS, dtype: int64

In [ ]:
df1 = df1.dropna()

In [ ]:
include_list = ['N','V','AJ','AD','P','PP','I','C']
df1 = df1[df1['TPS'].str.contains('|'.join(include_list))]

In [ ]:
('AD','ADJ')==('AD','V')

False

In [ ]:
df_group = df1.groupby(['QPS','TPS']).mean()
df_group = df_group.reset_index()


In [ ]:
d_weights = {}
for row in df_group.itertuples():
  d_weights[(row.QPS,row.TPS)]=row.FSG

In [ ]:
# d = {}
# for row in df1.itertuples():
#   # print(row.TPS)
#   if row.QPS not in d:
#     if row.TPS in ['N','AJ','V','AD','P','PP','I','C']: 
#       d[row.QPS] = [row.TPS]
#     else:
#       continue
#   elif row.TPS in ['N','AJ','V','AD','P','PP','I','C']: d[row.QPS].append(row.TPS)


In [ ]:
lst = ['N','AJ','V','AD','P','PP','I','C']

In [ ]:
G = nx.Graph()
for n in lst:
  G.add_node(n)
for row in df1.itertuples():
  if row.TPS in ['N','AJ','V','AD','P','PP','I','C']: 
    G.add_edge(row.QPS,row.TPS,weight = d_weights[(row.QPS,row.TPS)])

In [ ]:
G.nodes()

NodeView(('N', 'AJ', 'V', 'AD', 'P', 'PP', 'I', 'C'))

In [ ]:
nx.write_gml(G,'POS.gml')